In [ ]:
import pandas as pd 
import numpy as np 
from tqdm.auto import tqdm

import torch 
from torch import nn 

import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import seaborn as sns 
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split 
import requests
from pathlib import Path



# Download helper functions from Learn PyTorch repo
if Path("helper_functions.py").is_file():
  print("helper_functions.py already exists, skipping download...")
else:
  print("Downloading helper_functions.py")
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)
from helper_functions import accuracy_fn 

In [ ]:
train = pd.read_csv('../input/digit-recognizer/train.csv',dtype = np.float32)
test_data = pd.read_csv('../input/digit-recognizer/test.csv',dtype = np.float32)
sample_submission = pd.read_csv('../input/digit-recognizer/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
labels = train['label']
class_names = labels.unique()
y = train.label.values
X = train.drop(['label'],axis = 1)
X = X/255
X = X.values.reshape(-1,28,28,1)
f = train.loc[:,train.columns !="label"].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2,
                                                   random_state = 42)

In [ ]:
# need to convert to tensor 
print(type(X_train))
print(type(X_test))
print(type(y_train))
print(type(y_test))

In [ ]:
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train).type(torch.LongTensor)
X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test).type(torch.LongTensor)

In [ ]:
# need to convert to tensor 
print(type(X_train))
print(type(X_test))
print(type(y_train))
print(type(y_test))

In [ ]:
# Convert data to dataloader 
# 1. convert to en tensor dataset 
train = torch.utils.data.TensorDataset(X_train,y_train)
test = torch.utils.data.TensorDataset(X_test,y_test)

# Convert to dataloader 
BATCH_SIZE = 32
train_dataloader = DataLoader(train,BATCH_SIZE, shuffle = False)
test_dataloader = DataLoader(test,BATCH_SIZE, shuffle = False)

In [ ]:
# Let's check out what what we've created
print(f"Length of train_dataloader: {len(train_dataloader)} batches of {BATCH_SIZE}...")
print(f"Length of test_dataloader: {len(test_dataloader)} batches of {BATCH_SIZE}...")


In [ ]:
class Digits_model0(nn.Module):
    def __init__(self,input_shape : int, output_shape : int, hidden_units :int):
        super().__init__()
        
        self.layer_stack = nn.Sequential(
        nn.Flatten(),
        nn.Linear(input_shape, hidden_units),
        nn.Linear(hidden_units, output_shape))
    def forward (self,x): 
        return self.layer_stack(x)

In [ ]:
torch.manual_seed(42)
model_0 = Digits_model0(784,10,len(class_names))

In [ ]:
# TRAINING FUNCTION
def train_step(model: torch.nn.Module,
               data_loader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
              accuracy_fn):
    train_loss, train_acc = 0, 0
    
    model.train()
    for batch, (X,y) in enumerate(data_loader):
        y_pred = model(X)
        
        loss = loss_fn(y_pred,y)
        train_loss += loss
        train_acc += accuracy_fn(y, y_pred.argmax(dim =1))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    train_loss /= len(data_loader)
    train_acc /= len(data_loader)
    
    print(f"Train loss : {train_loss : .5f}, Train acc : {train_acc : .5f}%")
    
# TEST FUNCTION 
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader, 
              loss_fn: torch.nn.Module,
              accuracy_fn):
    test_loss, test_acc = 0,0 
    
    model.eval()
    
    with torch.inference_mode():
        for X,y in dataloader:
            test_pred = model(X)
            test_loss += loss_fn(test_pred,y)
            test_acc += accuracy_fn(y,test_pred.argmax(dim =1))
            
        test_loss /= len(dataloader)
        test_acc /= len(dataloader)
    print(f"Test loss : {test_loss : .5f}, Train acc : {test_acc : .5f}%")
        
# Move values to device
torch.manual_seed(42)
def eval_model(model: torch.nn.Module, 
               data_loader: torch.utils.data.DataLoader, 
               loss_fn: torch.nn.Module, 
               accuracy_fn):
    loss, acc = 0, 0
    model.eval()
    with torch.inference_mode():
        for X, y in data_loader:
            # Send data to the target device
            X, y = X, y
            y_pred = model(X)
            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(y_true=y, y_pred=y_pred.argmax(dim=1))
        
        loss /= len(data_loader)
        acc /= len(data_loader)
    return {"model_name": model.__class__.__name__, # only works when model was created with a class
            "model_loss": loss.item(),
            "model_acc": acc}

In [ ]:
torch.manual_seed(42)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model_0.parameters(),lr = 0.01)

epochs = 10 

for epoch in tqdm(range(epochs)):
    if epoch % 10 == 0 : 
        print(f"Epoch : {epoch} \n------")
        
    train_step(model_0,
              train_dataloader,
               loss_fn,
              optimizer,
              accuracy_fn)
    test_step(model_0,
             test_dataloader,
             loss_fn,
             accuracy_fn)


In [ ]:
model_0_results = eval_model(model=model_0, data_loader=test_dataloader,
    loss_fn=loss_fn, accuracy_fn=accuracy_fn
)
model_0_results

## Model 2 

In [ ]:
class Digits_Model2(nn.Module):
    def __init__(self,input_shape : int, output_shape : int, hidden_units : int):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(input_shape, hidden_units,kernel_size = 3, 
                     stride = 1, padding = 1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2)
        )
        
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units,hidden_units, kernel_size = 3, padding = 1, stride = 1 ),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, kernel_size = 2, padding = 1, stride = 1 ),
            nn.ReLU(),
            nn.MaxPool2d(2))
        
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(hidden_units*7*7, output_shape))
        
        
    def forward(self,x ):
        x = x.permute(0,3, 2, 1)
        x = self.block_1(x)
       #1 print(x.shape)
        x = self.block_2(x)
     
        x = self.classifier(x)
        #print(x.shape)
        return x 
        #return self.classifier(self.conv_bloc_2(self.conv_block_1(x))).unsqueeze(dim = 0 )

In [ ]:
torch.manual_seed(42)
model_2 = Digits_Model2(input_shape = 1,
                              hidden_units = 10,
                              output_shape = len(class_names))
model_2

In [ ]:
torch.manual_seed(42)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params = model_2.parameters(),lr = 0.01)

epochs = 20 

for epoch in tqdm(range(epochs)):
    if epoch % 10 == 0 : 
        print(f"Epoch : {epoch} \n------")
        
    train_step(model_2,
              train_dataloader,
               loss_fn,
              optimizer,
              accuracy_fn)
    test_step(model_2,
             test_dataloader,
             loss_fn,
             accuracy_fn)


In [ ]:
model_2_results = eval_model(model=model_2, data_loader=test_dataloader,
    loss_fn=loss_fn, accuracy_fn=accuracy_fn
)
model_2_results

In [ ]:
sample_submission.head()

In [ ]:
test_data = test_data/255
test_data

In [ ]:
test_data = test_data.values.reshape(-1,28,28,1)

In [ ]:
test_data = torch.from_numpy(test_data)

In [ ]:
prediction = model_2(test_data)

In [ ]:
my_pred = prediction.argmax(dim = 1).detach().numpy()

In [ ]:
results = pd.Series(my_pred,name = "Label")
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results], axis = 1 )

In [ ]:
submission.to_csv('submission.csv',index = False )

In [ ]:
submission